In [34]:
WORKER_COUNT = 8

In [2]:
# whether to log each feature and sequence status
verbose = True

In [31]:
import gc
import os
import pandas as pd
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

pd.options.display.max_rows = 200

In [4]:
# setup paths
pwd = os.getcwd().replace("deepvideoclassification","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [5]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [6]:
from deepvideoclassification.architectures import Architecture
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed

Using TensorFlow backend.


# Create experiments lists and run experiments

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths

* batch 3 = run trainable MLP and LRCNN on best performing frozen variants

* batch 4 = run trainable but initializing with best CNN weights

* batch 5 = run C3D models

# Batch 1

In [19]:
experiment_batch_name = 'experiment_batch_1'

In [20]:
# init list of experiments
experiments = []

In [21]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [22]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [23]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters

                        # add to list of experiments
                        experiments.append(experiment)

In [24]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers

                                # add to list of experiments
                                experiments.append(experiment)

In [25]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)
experiments['model_id'] = experiments.index

In [26]:
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [36]:
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [ ]:
###################
### Run experiments
###################

for row in experiments.values:
    
    # get experiment params from dataframe row
    experiment = dict(zip(experiments.columns, row))
    print(experiment)
    
    # only run experiment if not already run
    if not os.path.exists(pwd + 'models/' + str(model_id) + '/results.json'):

        logging.info("Begin experiment for model_id={}".format(experiment['model_id']))

        architecture = Architecture(model_id = experiment['model_id'], 
                                    architecture = experiment['architecture'], 
                                    sequence_length = experiment['sequence_length'], 
                                    pretrained_model_name = experiment['pretrained_model_name'],
                                    pooling = experiment['pooling'],
                                    sequence_model = experiment['sequence_model'],
                                    sequence_model_layers = experiment['sequence_model_layers'],
                                    layer_1_size = experiment['layer_1_size'],
                                    layer_2_size = experiment['layer_2_size'],
                                    layer_3_size = experiment['layer_3_size'],
                                    dropout = experiment['dropout'],
                                    verbose=True)

        architecture.train_model()

# Batch 2

In [35]:
# for best configurations from batch 1, run other pretrained models 
# in buckets and run longer sequence lengths

# Batch 3

In [ ]:
# run trainable MLP and LRCNN on best performing frozen variants

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

# Batch 4

In [ ]:
# run trainable but initializing with best CNN weights

# Batch 5

In [ ]:
#######
### C3D
#######

architecture = 'C3D'

############
### C3Dsmall
############

architecture = 'C3Dsmall' 

# Analyze results

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [ ]:
results.head(10)